![](module_2_flowsheet.png) 

In [2]:
from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)
from pyomo.network import Arc, SequentialDecomposition

In [3]:
from idaes.core import FlowsheetBlock
from idaes.unit_models import (PressureChanger,
                               Mixer,
                               Separator as Splitter,
                               Heater,
                               StoichiometricReactor)
from idaes.unit_models import Flash
from idaes.unit_models.pressure_changer import ThermodynamicAssumption
import hda_ideal_VLE as thermo_props
import hda_reaction as reaction_props

In [4]:
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})
m.fs.thermo_params = thermo_props.HDAParameterBlock()
m.fs.reaction_params = reaction_props.HDAReactionParameterBlock(
        default={"property_package": m.fs.thermo_params})

In [5]:
m.fs.M101 = Mixer(default={"property_package": m.fs.thermo_params,
                           "inlet_list": ["toluene_feed", "hydrogen_feed", "vapor_recycle"]})

m.fs.H101 = Heater(default={"property_package": m.fs.thermo_params,
                            "has_pressure_change": False,
                            "has_phase_equilibrium": True})
m.fs.R101 = StoichiometricReactor(
            default={"property_package": m.fs.thermo_params,
                     "reaction_package": m.fs.reaction_params,
                     "has_heat_of_reaction": True,
                     "has_heat_transfer": True,
                     "has_pressure_change": False})
m.fs.F101 = Flash(default={"property_package": m.fs.thermo_params,
                               "has_heat_transfer": True,
                               "has_pressure_change": True})
m.fs.S101 = Splitter(default={"property_package": m.fs.thermo_params,
                               "ideal_separation": False,
                               "outlet_list": ["purge", "recycle"]})
    

m.fs.C101 = PressureChanger(default={
            "property_package": m.fs.thermo_params,
            "compressor": True,
            "thermodynamic_assumption": ThermodynamicAssumption.isothermal})
    
m.fs.F102 = Flash(default={"property_package": m.fs.thermo_params,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})

In [6]:
m.fs.s03 = Arc(source=m.fs.M101.outlet, destination=m.fs.H101.inlet)
m.fs.s04 = Arc(source=m.fs.H101.outlet, destination=m.fs.R101.inlet)
m.fs.s05 = Arc(source=m.fs.R101.outlet, destination=m.fs.F101.inlet)
m.fs.s06 = Arc(source=m.fs.F101.vap_outlet, destination=m.fs.S101.inlet)
m.fs.s08 = Arc(source=m.fs.S101.recycle, destination=m.fs.C101.inlet)
m.fs.s09 = Arc(source=m.fs.C101.outlet,
               destination=m.fs.M101.vapor_recycle)
m.fs.s10 = Arc(source=m.fs.F101.liq_outlet, destination=m.fs.F102.inlet)

In [7]:
from idaes.core import UnitModelBlockData
from pyomo.network.port import SimplePort
from pyomo.network.arc import SimpleArc

import json

In [8]:
comps = m.component_objects(descend_into=False)
flowsheet = next(comps)
ports = {}

unit_models = {}
arcs = []
for component in flowsheet.component_objects(descend_into=False):
    if isinstance(component, UnitModelBlockData):
        unit_models[component] = {"name": component.getname(), "type": type(component).__name__}
        
        for subcomponent in component.component_objects(descend_into=False):
            if isinstance(subcomponent, SimplePort):
                ports[subcomponent] = component
                
    elif isinstance(component, SimpleArc):
       arcs.append(component)
    
edges = {}
orphaned_ports = set(ports.keys())
for arc in arcs:
    edges[(ports[arc.source], ports[arc.dest])] = arc
    orphaned_ports.discard(arc.source)
    orphaned_ports.discard(arc.dest)
    

In [9]:
from collections import defaultdict

labeled_edges = defaultdict(list)

for (source, dest) in edges:
    labeled_edges[source.getname()].append(dest.getname())
    
for port in orphaned_ports:
    labeled_edges["Orphaned"].append(ports[port].getname())

with open('edges.json', 'w') as outfile:  
    json.dump(labeled_edges, outfile)
    
named_components = {}
for comp in unit_models.values():
    named_components[comp["name"]]= {"type": comp["type"]}
    
with open('nodes.json', 'w') as outfile:  
    json.dump(named_components, outfile)